In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

try:
    pdf_path = "data/Endreport.pdf"   # your actual file path
    pymupdf_loader = PyMuPDFLoader(pdf_path)
    pymupdf_docs = pymupdf_loader.load()

    print(f" Loaded {len(pymupdf_docs)} pages")
    print(" Includes detailed metadata")
    print(pymupdf_docs[:2])   # show first 2 pages as preview

except Exception as e:
    print(f" Error: {e}")


c:\Users\shett\Project by Ashish shetty\Ultimate Rag Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Loaded 29 pages
 Includes detailed metadata
[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': '2025-05-27T08:19:51+00:00', 'source': 'data/Endreport.pdf', 'file_path': 'data/Endreport.pdf', 'total_pages': 29, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-20T10:25:20-05:00', 'trapped': '', 'modDate': "D:20251020102520-05'00'", 'creationDate': 'D:20250527081951Z', 'page': 0}, page_content='Online Banking\nOnline Banking Service Agreement\nBank of America Online Banking Service Agreement\nEffective Date: July 21, 2025\nTable of Contents: Hide all Topics\n1. General Description of Bank of America Online Banking Service Agreement (this "Agreement")\nIntroduction\nA. What This Agreement Covers\nB. Accepting the Agreement\nC. Relation to Other Agreements\n2. Payment & Transfer Services Using Internal Accounts and Payments to Your Bank of America Loan Accounts From An External Account\nIntroduction\nA. Payment &

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # adjust as needed
        chunk_overlap=150,       # overlap for context
        separators=["\n\n", "\n", " ", ""]
    )

In [4]:
chunks = text_splitter.split_documents(pymupdf_docs)
print(f" Total chunks created: {len(chunks)}")
print(" Preview first 2 chunks:\n")
print(chunks[:2])


 Total chunks created: 153
 Preview first 2 chunks:

[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': '2025-05-27T08:19:51+00:00', 'source': 'data/Endreport.pdf', 'file_path': 'data/Endreport.pdf', 'total_pages': 29, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-20T10:25:20-05:00', 'trapped': '', 'modDate': "D:20251020102520-05'00'", 'creationDate': 'D:20250527081951Z', 'page': 0}, page_content='Online Banking\nOnline Banking Service Agreement\nBank of America Online Banking Service Agreement\nEffective Date: July 21, 2025\nTable of Contents: Hide all Topics\n1. General Description of Bank of America Online Banking Service Agreement (this "Agreement")\nIntroduction\nA. What This Agreement Covers\nB. Accepting the Agreement\nC. Relation to Other Agreements\n2. Payment & Transfer Services Using Internal Accounts and Payments to Your Bank of America Loan Accounts From An External Account\nIntroduction\nA. P

In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

try:
    print(f"Total chunks ready for embedding: {len(chunks)}")

    model_name = "BAAI/bge-m3"

    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"},          # change to cuda for GPU
        encode_kwargs={"normalize_embeddings": True}
    )

    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local("faiss_bank_terms_bge_m3")

    print("Embedding + FAISS index created successfully.")
    print("Index saved to: faiss_bank_terms_bge_m3")

except Exception as e:
    print(f"Error: {e}")


Total chunks ready for embedding: 153


C:\Users\shett\AppData\Local\Temp\ipykernel_12524\3607747356.py:9: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


Embedding + FAISS index created successfully.
Index saved to: faiss_bank_terms_bge_m3


In [6]:
sample_text="RAG is SO interesting!"
vector=embeddings.embed_query(sample_text)
len(vector)

1024

In [7]:
chunks

[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': '2025-05-27T08:19:51+00:00', 'source': 'data/Endreport.pdf', 'file_path': 'data/Endreport.pdf', 'total_pages': 29, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-20T10:25:20-05:00', 'trapped': '', 'modDate': "D:20251020102520-05'00'", 'creationDate': 'D:20250527081951Z', 'page': 0}, page_content='Online Banking\nOnline Banking Service Agreement\nBank of America Online Banking Service Agreement\nEffective Date: July 21, 2025\nTable of Contents: Hide all Topics\n1. General Description of Bank of America Online Banking Service Agreement (this "Agreement")\nIntroduction\nA. What This Agreement Covers\nB. Accepting the Agreement\nC. Relation to Other Agreements\n2. Payment & Transfer Services Using Internal Accounts and Payments to Your Bank of America Loan Accounts From An External Account\nIntroduction\nA. Payment & Transfer Services Using Internal Accounts\nB

In [8]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "faiss_bank_terms_bge_m3",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 153 vectors


In [9]:
# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}   # you can adjust k if needed
)

print("Retriever created successfully!")


Retriever created successfully!


In [10]:
query = "What is Zelle service?"
results = retriever.invoke(query)

for i, doc in enumerate(results, start=1):
    print(f"\nChunk {i}:")
    print(doc.page_content[:300], "...")



Chunk 1:
4. Zelle® Network Service
A. Description of Service
We have partnered with the Zelle  Network to enable a convenient way to send and receive money with others you trust who are enrolled in
Zelle  through Bank of America or with another financial institution that partners with Zelle  (each, a “User”) ...

Chunk 2:
C. E-Bills
D. Limits
E. Canceling Bill Payments
F. Fees
4. Zelle® Network Service
Introduction
A. Description of Service
B. Eligibility and User Profile
C. Enrolling for the Service
D. Consent to Emails and Automated Text Messages
E. Receiving Money; Money Transfers by Network Banks
F. Sending Money ...

Chunk 3:
expose Bank of America or Zelle  to risk or liability, or we believe, in our sole discretion, that you have otherwise violated the terms and conditions
of using this Service.
We may determine other eligibility criteria in our sole discretion.
The Service may include functionality to add a unique alp ...

Chunk 4:
Zelle tags, both we and Zelle  have absolute 

In [12]:
import os
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model
os.environ["GROK_API_KEY"]=os.getenv("GROK_API_KEY")

llm = ChatGroq(model="llama-3.1-8b-instant", api_key=os.getenv("GROK_API_KEY"))
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017BA79981A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017BA7999400>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
llm.invoke("Hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.007236294, 'completion_tokens_details': None, 'prompt_time': 0.001629177, 'prompt_tokens_details': None, 'queue_time': 0.053851163, 'total_time': 0.008865471}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f036665b-3a7a-44be-9c2e-d2ce1cba8be1-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44})

In [16]:
from langchain_core.prompts import ChatPromptTemplate

simple_prompt = ChatPromptTemplate.from_template("""
Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:
""")


In [17]:
import textwrap
from typing import List
source_file_path = "/mnt/data/Endreport.pdf"

In [18]:
def build_context(docs: List, max_chars: int = 4000) -> str:
    pieces = []
    total = 0
    for d in docs:
        page = d.metadata.get("page", None) if hasattr(d, "metadata") else None
        src = d.metadata.get("source", source_file_path) if hasattr(d, "metadata") else source_file_path
        citation = f"[source: {src} page:{page}]" if page is not None else f"[source: {src}]"

        text = d.page_content.strip().replace("\n", " ")
        snippet = textwrap.shorten(text, width=1000, placeholder=" ...")
        chunk = f"{citation}\n{snippet}\n"
        pieces.append(chunk)

        total += len(chunk)
        if total >= max_chars:
            break

    return "\n\n".join(pieces)

In [19]:
def rag_answer(query: str, k: int = 5, max_context_chars: int = 4000):
    retrieved_docs = retriever.invoke(query)
    retrieved_docs = retrieved_docs[:k]
    context = build_context(retrieved_docs, max_chars=max_context_chars)
    prompt_text = simple_prompt.format(context=context, question=query)
    response = llm.invoke(prompt_text)
    return {
        "query": query,
        "answer": response,          
        "context": context,          
        "retrieved": retrieved_docs   
    }


In [20]:
out = rag_answer("What is Zelle service?")
print("=== ANSWER ===")
print(out["answer"])
print("\n=== CONTEXT SENT TO LLM (truncated) ===")
print(out["context"][:1500], "...\n")
print("\n=== TOP 3 RETRIEVED CHUNKS (citations + preview) ===")
for i, d in enumerate(out["retrieved"][:3], start=1):
    page = d.metadata.get("page", "unknown")
    src = d.metadata.get("source", source_file_path)
    print(f"\n-- CHUNK {i} -- {src} page:{page}")
    print(d.page_content[:800].strip(), "...\n")

=== ANSWER ===
content='The Zelle service is a convenient way to send and receive money with others who are enrolled in Zelle through Bank of America or with another financial institution that partners with Zelle, using aliases such as email addresses, U.S. mobile phone numbers, or other unique identifiers.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 1026, 'total_tokens': 1082, 'completion_time': 0.064046343, 'completion_tokens_details': None, 'prompt_time': 0.064380284, 'prompt_tokens_details': None, 'queue_time': 0.054704826, 'total_time': 0.128426627}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--274dee6b-6b0a-41cb-b2a6-59ae0951c682-0' usage_metadata={'input_tokens': 1026, 'output_tokens': 56, 'total_tokens': 1082}

=== CONTEXT SENT TO LLM (truncated) ===
[source: data/Endreport.pdf page:8

In [21]:
out = rag_answer("Can the bank change this agreement?")
print("=== ANSWER ===")
print(out["answer"])
print("\n=== CONTEXT SENT TO LLM (truncated) ===")
print(out["context"][:1500], "...\n")
print("\n=== TOP 3 RETRIEVED CHUNKS (citations + preview) ===")
for i, d in enumerate(out["retrieved"][:3], start=1):
    page = d.metadata.get("page", "unknown")
    src = d.metadata.get("source", source_file_path)
    print(f"\n-- CHUNK {i} -- {src} page:{page}")
    print(d.page_content[:800].strip(), "...\n")

=== ANSWER ===
content='Yes, the bank can change this agreement at any time. According to page 26 of the Endreport.pdf, Bank of America may add, delete or change the terms of this Agreement at any time.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 933, 'total_tokens': 975, 'completion_time': 0.054092346, 'completion_tokens_details': None, 'prompt_time': 0.073581689, 'prompt_tokens_details': None, 'queue_time': 0.0528025, 'total_time': 0.127674035}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--cb4cdaca-4448-42b9-a696-3f24037d5065-0' usage_metadata={'input_tokens': 933, 'output_tokens': 42, 'total_tokens': 975}

=== CONTEXT SENT TO LLM (truncated) ===
[source: data/Endreport.pdf page:17]
you no longer have the right to cancel it. Bank of America may at its option accept your cancellations or ame

In [ ]:
out = rag_answer("When was Bank of America founded?")
print("=== ANSWER ===")
print(out["answer"])
print("\n=== CONTEXT SENT TO LLM (truncated) ===")
print(out["context"][:1500], "...\n")
print("\n=== TOP 3 RETRIEVED CHUNKS (citations + preview) ===")
for i, d in enumerate(out["retrieved"][:3], start=1):
    page = d.metadata.get("page", "unknown")
    src = d.metadata.get("source", source_file_path)
    print(f"\n-- CHUNK {i} -- {src} page:{page}")
    print(d.page_content[:800].strip(), "...\n")

=== ANSWER ===
content='The provided context does not mention the founding date of Bank of America.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 913, 'total_tokens': 928, 'completion_time': 0.020705138, 'completion_tokens_details': None, 'prompt_time': 0.074359401, 'prompt_tokens_details': None, 'queue_time': 0.050764058, 'total_time': 0.095064539}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--c0f97644-af14-43a8-bba2-93e022fde00d-0' usage_metadata={'input_tokens': 913, 'output_tokens': 15, 'total_tokens': 928}

=== CONTEXT SENT TO LLM (truncated) ===
[source: data/Endreport.pdf page:28]
from the Website via desktop, tablet or mobile device. Business Services Addendum Some accounts and services, and the fees that apply to them, vary from state to state. Please review the information for your st